In [1]:
import pandas
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.spatial


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [4]:
from google.colab import files
uploaded = files.upload()

Saving AAPL_combine_updated.csv to AAPL_combine_updated.csv


In [5]:


def processText(column,n):#to remove stop words,to convert into lowercase and root word
    import re

    corpus = []
    for i in range(n):
        review = re.sub('[^a-zA-Z]', ' ', column[i])
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        all_stopwords = stopwords.words('english')
        all_stopwords.remove('not')
        review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus


In [6]:
def embed(input):
  return model(input)

In [7]:


df = pandas.read_csv('AAPL_combine_updated.csv')


df=df.dropna()
df = df[~(df['datetime'] >= '2023-01-01')]#dropping rows of year=2023

df=df.reset_index(drop=True)


print(df.columns)

titleColumn=df['title']
summaryColumn=df['summary']
dateColumn=df['datetime']
titleColumn=list(titleColumn)
summaryColumn=list(summaryColumn)

corpus=processText(titleColumn,len(titleColumn))#preprocessing title column
df['title']=corpus

summaryCorpus=processText(summaryColumn,len(summaryColumn))#preprocessing summary column
df['summary']=summaryCorpus


df = df.astype({'title':'string'})
df = df.astype({'summary':'string'})

print(df.tail(5))

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'y_actual', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'summary', 'title'],
      dtype='object')
       datetime       close        open         low        high      volume  \
969  2022-12-22  132.028412  134.350006  130.300003  134.559998  77852100.0   
970  2022-12-23  131.658981  130.919998  129.639999  132.419998  63814900.0   
971  2022-12-27  129.831772  131.380005  128.720001  131.410004  69007800.0   
972  2022-12-28  125.847855  129.669998  125.870003  131.029999  85438400.0   
973  2022-12-29  129.412415  127.989998  127.730003  130.479996  75703700.0   



In [8]:
df.shape

(974, 29)

In [9]:
titleColumn=df['title']
titleColumn[0]

# df=df.fillna(0)

# df=df.dropna()

'avan motor line six electr scooter launch one everi month moneycontrol mac pro imac qualcomm expect appl start appleinsid crorepati portfolio could turn rs monthli sip rs crore moneycontrol china didi launch credit crowdfund servic diversif reuter pro con buy appl aapl stock wtop perform dow compon nysearca dia seek alpha set vip mail contact iphon ipad io appleinsid ipad iphon xr lead christma activ help xr catch xs market share appleinsid holiday adventur maco mojav mac observ fool first icloud phish scam year idrop news beat dj ebro darden appoint appl music global editori head cover hip hop r b appleinsid juici year histori new york fame cartier mansion bloomberg usb type c protocol similar mfi final could block low qualiti charger appleinsid syndic bank sbi life insur sign bancassur pact moneycontrol stock watch china talk tough taiwan realmoney japanes yen soar flash crash sweep currenc market reuter meet year old ghanaian tech geniu code uber instagram snapchat puls ghana revie

In [10]:
summaryColumn=df['summary']
summaryColumn[0]

'oil rich saudi arabia see invest opportun usd billion india variou sector said visit crown princ moham bin salman wednesday naddress media along prime minist narendra modi crown princ said saudi arabia alreadi invest usd billion india sinc visit modi gulf nation nhe also laud india strength inform technolog said saudi arabia invest lot sector n understand opportun usd billion invest india want strengthen invest econom tie ensur use return countri crown princ said nhe inform saudi arabia invest india petrochem diversifi sector toggl dark modewhen john pfaff found appl iie comput sit parent attic year expect still work order nand appl iie younger probabl nostalgia trip nan appl iie npic twitter com zl wwxoo john pfaff johnfpfaff februari amazingli appl iie bootabl npic twitter com ivet l john pfaff johnfpfaff februari appl iie comput first releas back appl desktop sold macintosh brand name stock market quiet wednesday differ benchmark react varieti news event nsever stock post strong ad

In [11]:
df.head(5)

,datetime,close,open,low,high,volume,y_actual,evm,force_index,rsi,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,summary,title
0,2019-02-20,41.624268,42.797501,42.747501,43.330002,104457600.0,0.0,0.097372,1.891971e+08,62.386241,...,69391000.0,45896000.0,-90976000.0,58896000.0,338516000.0,248028000.0,90488000.0,182295000.0,oil rich saudi arabia see invest opportun usd ...,avan motor line six electr scooter launch one ...
1,2019-02-21,41.389565,42.950001,42.575001,43.092499,68998800.0,1.0,0.042723,8.899667e+07,60.091586,...,69391000.0,45896000.0,-90976000.0,58896000.0,338516000.0,248028000.0,90488000.0,182295000.0,live bse livens live volum today l h shreesh b...,outlier quick view varun beverag moneycontrol ...
2,2019-02-22,41.851711,42.895000,42.845001,43.250000,75652800.0,1.0,0.043863,1.310842e+08,62.979083,...,69391000.0,45896000.0,-90976000.0,58896000.0,338516000.0,248028000.0,90488000.0,182295000.0,piram enterpris pel largest sell painkil sarid...,saridon pain relief tablet sold everi second s...
3,2019-02-25,42.156578,43.540001,43.487499,43.967499,87493600.0,1.0,0.039235,7.856815e+07,64.788967,...,69391000.0,45896000.0,-90976000.0,58896000.0,338516000.0,248028000.0,90488000.0,182295000.0,competit tough victoria secret lead substanti ...,appl surviv iphon challeng like microsoft appl...
4,2019-02-26,42.180771,43.427502,43.292500,43.825001,68280800.0,1.0,-0.005992,1.476660e+07,64.935466,...,69391000.0,45896000.0,-90976000.0,58896000.0,338516000.0,248028000.0,90488000.0,182295000.0,happen npleas make sure browser support javasc...,rais bet tencent otcmkt tcehi seek alpha appl ...


In [12]:
df.tail(5)

,datetime,close,open,low,high,volume,y_actual,evm,force_index,rsi,...,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,summary,title
969,2022-12-22,132.028412,134.350006,130.300003,134.559998,77852100.0,0.0,-5.483888,-1.211086e+09,36.101057,...,122151000.0,-22354000.0,-110749000.0,111443000.0,352755000.0,302083000.0,50672000.0,149631000.0,crypto winter damn bitcoin ethereum investor s...,bitcoin interest surg crypto winter arriv morn...
970,2022-12-23,131.658981,130.919998,129.639999,132.419998,63814900.0,0.0,-6.731920,-9.411091e+08,35.694640,...,122151000.0,-22354000.0,-110749000.0,111443000.0,352755000.0,302083000.0,50672000.0,149631000.0,dec reuter appl inc aapl appl watch electrocar...,appl watch violat alivecor patent import ban h...
971,2022-12-27,129.831772,131.380005,128.720001,131.410004,69007800.0,0.0,-4.783137,-8.874656e+08,33.675324,...,122151000.0,-22354000.0,-110749000.0,111443000.0,352755000.0,302083000.0,50672000.0,149631000.0,last month insid transact gn store nordin fact...,merri christma us appleinsid appleinsid best a...
972,2022-12-28,125.847855,129.669998,125.870003,131.029999,85438400.0,1.0,-4.469965,-1.271092e+09,29.726630,...,122151000.0,-22354000.0,-110749000.0,111443000.0,352755000.0,302083000.0,50672000.0,149631000.0,appl expect launch buy pay later servic call a...,catalyst appl stock first major new product be...
973,2022-12-29,129.412415,127.989998,127.730003,130.479996,75703700.0,1.0,-4.473906,-9.856706e+08,36.860470,...,122151000.0,-22354000.0,-110749000.0,111443000.0,352755000.0,302083000.0,50672000.0,149631000.0,led tech investor much cautiou opt make switch...,netflix lead faang higher could fail thestreet...


In [13]:
df.columns

Index(['datetime', 'close', 'open', 'low', 'high', 'volume', 'y_actual', 'evm',
       'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
       'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',
       'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
       'Free Cash Flow', 'Total Assets',
       'Total Liabilities Net Minority Interest',
       'Total Equity Gross Minority Interest', 'Total Capitalization',
       'summary', 'title'],
      dtype='object')

In [14]:
# n,m=df.shape#975 27
# daily = df['close'].shift(-1)/df['close'] - 1
# print(type(daily))
# # daily=daily.fillna(0)
# # print(daily.isnull().sum())

# y = np.zeros(n)


In [15]:
# for i in range(n):
#   if daily[i] >= 0:
#     y[i] = 1 # up
#   else:
#     y[i] = 0 # down
# print(len(y))
# y = np.insert(y, 0, 0)
# y=y[:-1]
# print(len(y))
# headers = ['y_lables']
# y_df = pd.DataFrame(y, columns=headers)
# y_df['datetime']=df['datetime']
# y_df.head(5)

In [16]:
df['y_actual']

0      0.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
969    0.0
970    0.0
971    0.0
972    1.0
973    1.0
Name: y_actual, Length: 974, dtype: float64

In [17]:
y=df['y_actual']
# y=np.array(y)
y

0      0.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
969    0.0
970    0.0
971    0.0
972    1.0
973    1.0
Name: y_actual, Length: 974, dtype: float64

In [18]:
column_names=["y_actual"]

# Create DataFrame by assigning column names
y_df=pd.DataFrame(y, columns=column_names)

In [19]:
y_df

,y_actual
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
969,0.0
970,0.0
971,0.0
972,1.0


In [20]:
# headers = ['y_lables']
# y_df = pd.DataFrame(y, columns=headers)
y_df['datetime']=df['datetime']

In [21]:
y_df.head(5)

,y_actual,datetime
0,0.0,2019-02-20
1,1.0,2019-02-21
2,1.0,2019-02-22
3,1.0,2019-02-25
4,1.0,2019-02-26


In [22]:
y_df.shape

(974, 2)

In [23]:
y_df.tail(5)

,y_actual,datetime
969,0.0,2022-12-22
970,0.0,2022-12-23
971,0.0,2022-12-27
972,1.0,2022-12-28
973,1.0,2022-12-29


In [24]:
# df_title=df[['datetime','title']]
df=df.drop(['datetime','title','summary','y_actual'],axis='columns')#dropping the date column


In [25]:
scalar = StandardScaler()
scaled_df = scalar.fit_transform(df) #scaling the csv data

df=scaled_df


In [26]:
df.shape

(974, 25)

In [27]:

text_vector=[]# will contain vector representation of titleColumn
summary_vector=[]

In [28]:
for text in titleColumn:
    sentence = text
    message_embeddings = embed([sentence])
    temp=np.array(message_embeddings[0])
    temp=temp.T
    text_vector.append(temp)


In [29]:
for summary in summaryColumn:
    sentence = summary
    message_embeddings = embed([sentence])
    temp=np.array(message_embeddings[0])
    temp=temp.T
    summary_vector.append(temp)

In [30]:
text_vector=np.array(text_vector)
summary_vector=np.array(summary_vector)

In [31]:
text_vector.shape

(974, 512)

In [32]:
summary_vector.shape

(974, 512)

In [33]:

scalar = StandardScaler()
scaled_data_text = scalar.fit_transform(text_vector) #scaling the data


In [34]:
scalar = StandardScaler()
scaled_data_summary = scalar.fit_transform(summary_vector) #scaling the data

In [35]:
pca = PCA(n_components = 25)
pca.fit(scaled_data_text)
data_pca_text = pca.transform(scaled_data_text)
data_pca_text

array([[22.654774  , 19.68477   ,  7.1595306 , ..., -4.726308  ,
         3.2850368 ,  5.298541  ],
       [ 1.5638537 , -3.473331  , -3.7826233 , ..., -1.0812197 ,
         2.0129745 ,  2.057868  ],
       [-9.1792965 ,  2.8117106 , -5.913816  , ..., -0.32488275,
         0.07076359,  2.414145  ],
       ...,
       [19.57706   , 10.722677  ,  2.962521  , ...,  1.948776  ,
         0.68935764, -1.544311  ],
       [ 1.831466  , -7.1388097 ,  5.7734194 , ...,  1.9756095 ,
         0.7724328 , -2.2311869 ],
       [ 5.774376  , -9.416828  ,  3.0518963 , ...,  0.85809696,
         2.263319  , -1.5072662 ]], dtype=float32)

In [36]:
pca = PCA(n_components = 25)
pca.fit(scaled_data_summary)
data_pca_summary = pca.transform(scaled_data_summary)
data_pca_summary

array([[-4.0974379e+00, -5.0511160e+00,  9.2467270e+00, ...,
        -1.3215301e+00,  4.4715166e-02,  1.3527788e+00],
       [-6.0066757e+00, -4.6314211e+00,  6.6807365e+00, ...,
        -5.2633393e-01, -1.1545998e-01, -1.4741814e+00],
       [-6.4462938e+00, -1.5700805e+00, -1.1642354e+00, ...,
         5.6907296e-01,  2.6742041e-01,  5.7089001e-02],
       ...,
       [-5.8146448e+00, -1.9808619e+00,  3.5983148e-01, ...,
        -1.6400170e+00, -7.3881727e-01,  8.4864616e-01],
       [-6.6958251e+00, -3.6016133e+00,  2.8661485e+00, ...,
         1.9204798e-01,  3.9414695e-01,  1.8608484e-02],
       [-6.6291943e+00, -2.8937340e+00,  1.3680781e+00, ...,
         1.1837969e+00, -4.4012964e-03, -9.8158485e-01]], dtype=float32)

In [37]:
data_pca_text.shape

(974, 25)

In [38]:
data_pca_summary.shape

(974, 25)

In [39]:
# text_vector_dataframe=pd.DataFrame()
type(data_pca_text)
# df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
mycolumns=['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8','pca9','pca10','pca11','pca12','pca13','pca14','pca15','pca16','pca17','pca18','pca19','pca20','pca21','pca22','pca23','pca24','pca25']
df_news_text=pd.DataFrame(data_pca_text, columns = mycolumns)

In [40]:
# text_vector_dataframe=pd.DataFrame()
type(data_pca_summary)
# df_news  =pd.DataFrame(((x,) for x in data_pca), columns=['title'])
mycolumns=['Spca1','Spca2','Spca3','Spca4','Spca5','Spca6','Spca7','Spca8','Spca9','Spca10','Spca11','Spca12','Spca13','Spca14','Spca15','Spca16','Spca17','Spca18','Spca19','Spca20','Spca21','Spca22','Spca23','Spca24','Spca25']
df_news_summary=pd.DataFrame(data_pca_summary, columns = mycolumns)

In [41]:
df_news_text

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca16,pca17,pca18,pca19,pca20,pca21,pca22,pca23,pca24,pca25
0,22.654774,19.684771,7.159531,3.603495,-6.838140,10.428587,-3.448949,6.120075,-1.294774,3.576098,...,-5.884901,7.498806,6.223081,-0.133425,-4.311012,-2.451073,9.751688,-4.726308,3.285037,5.298541
1,1.563854,-3.473331,-3.782623,-2.508502,-4.720814,1.674747,-4.951632,1.372158,0.562664,-2.432447,...,2.336582,1.624870,-3.441215,2.344890,-0.679703,2.300196,3.617532,-1.081220,2.012975,2.057868
2,-9.179296,2.811711,-5.913816,2.304858,-0.152320,1.853811,5.503058,-3.226789,-0.228151,-2.627688,...,1.694156,4.000993,-3.171506,2.159171,0.398422,-0.849247,-1.344516,-0.324883,0.070764,2.414145
3,9.169606,-2.382734,-1.226832,-3.851335,-2.208233,-3.531028,2.682847,2.726148,-1.379948,-1.851651,...,-0.456782,2.354859,-0.507906,0.181548,1.894903,-0.502952,-1.094769,-4.337394,0.208617,2.613868
4,0.687009,-3.714859,-5.819294,-0.615771,-4.170208,-0.667097,0.414284,-0.899177,2.319784,1.319087,...,-0.003716,4.049840,-2.685757,-0.060991,-2.502755,0.822862,1.103148,-0.816562,1.442738,1.051238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,1.655340,-8.504667,4.405375,-0.847841,-0.786364,0.302262,-0.735276,0.922597,2.524419,-1.151824,...,-1.274392,0.681736,-0.204180,0.265542,-0.919507,-1.737674,0.853302,-0.736642,-0.459571,-0.771534
970,7.225430,-8.447567,0.959062,-1.203224,1.661753,3.582793,0.565506,-1.327860,1.392817,0.685632,...,1.539603,0.964195,-0.464519,1.898905,-1.834263,-1.860763,0.770940,0.618487,1.777401,-0.459317
971,19.577061,10.722677,2.962521,0.333361,-1.239834,-0.534513,1.546886,-3.370409,-0.934301,-0.480977,...,2.078183,-3.787397,-0.221134,0.543944,-0.113350,0.515963,-1.029364,1.948776,0.689358,-1.544311
972,1.831466,-7.138810,5.773419,0.343198,0.622320,5.114515,0.933785,-2.884283,-0.926939,-0.179519,...,3.556339,2.177893,0.769836,0.450071,0.577868,-0.536674,-0.230359,1.975610,0.772433,-2.231187


In [42]:
df_news_summary

,Spca1,Spca2,Spca3,Spca4,Spca5,Spca6,Spca7,Spca8,Spca9,Spca10,...,Spca16,Spca17,Spca18,Spca19,Spca20,Spca21,Spca22,Spca23,Spca24,Spca25
0,-4.097438,-5.051116,9.246727,-3.664545,4.503671,3.414853,-2.192408,-0.816809,-0.655531,-0.113480,...,1.818383,-0.914117,-2.074325,0.913988,0.112712,1.926086,0.260297,-1.321530,0.044715,1.352779
1,-6.006676,-4.631421,6.680737,-1.282974,1.987336,-0.930859,1.333789,0.147976,-0.838744,2.233258,...,1.650529,-0.181376,0.587466,-1.511634,1.054598,-1.483433,-1.453117,-0.526334,-0.115460,-1.474181
2,-6.446294,-1.570081,-1.164235,1.155925,-2.105601,1.013681,1.973031,-0.653831,-0.393226,-2.480187,...,-0.941116,0.429186,-1.367047,0.464688,-0.060432,0.348901,0.847478,0.569073,0.267420,0.057089
3,-6.669613,-3.886621,3.157756,-0.066977,-0.683434,-0.402385,1.849448,-0.358618,-0.574249,-0.090964,...,0.295991,-0.981859,0.993470,-1.462695,-0.706416,-0.579468,0.472940,0.470021,0.762850,-1.404587
4,-6.065101,-4.833769,4.544207,-0.517819,2.771022,-1.551171,-1.458420,-0.127880,-0.668664,0.083365,...,1.060050,0.447202,-2.514552,2.650107,-2.557092,-0.502712,2.301711,0.643496,-0.231621,-0.870302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,-5.741839,1.578586,-4.794973,1.394566,-1.604301,-0.661164,2.360312,0.755894,-1.045721,1.970019,...,1.727583,2.827416,1.035260,-1.005180,-0.452188,1.798969,-0.634948,-1.344085,-0.228949,-2.238541
970,-5.849784,-5.012734,6.648133,-1.643568,4.144670,0.928861,-1.634946,-0.966670,-0.288928,-0.395151,...,2.306204,-2.243432,-0.066597,1.084826,-2.327940,-1.495850,0.988852,-0.155374,0.065605,1.730080
971,-5.814645,-1.980862,0.359831,0.751724,-2.832372,-0.580992,1.352487,1.028613,2.240692,0.896156,...,0.873453,-2.523095,-0.288524,-1.318568,0.659262,-0.309576,1.374979,-1.640017,-0.738817,0.848646
972,-6.695825,-3.601613,2.866148,-0.293933,-1.249721,0.684557,0.608598,-0.053866,2.054352,-0.328348,...,0.050719,-2.594426,0.332053,-0.011782,-1.045049,-0.123244,2.000252,0.192048,0.394147,0.018608


In [43]:
# df=df.drop(['y_actual'],axis='columns')

In [44]:
# df.columns

In [45]:
column=['close', 'open', 'low', 'high', 'volume','evm',  'force_index', 'rsi', 'cci', 'macd-signal', 'atr', 'vwap', 'stx_7_3',
  'Total Revenue', 'Cost of Revenue', 'Gross Profit', 'Normalized EBITDA',  'Operating Cash Flow', 'Investing Cash Flow', 'Financing Cash Flow',
   'Free Cash Flow', 'Total Assets',  'Total Liabilities Net Minority Interest', 'Total Equity Gross Minority Interest', 'Total Capitalization']

df=pd.DataFrame(df,columns=column)
    #  'volume', 'y_actual', 'evm',
   

In [46]:
# df=df.drop(['y_actual'],axis='columns')

In [47]:
df.head(5)

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Gross Profit,Normalized EBITDA,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,Free Cash Flow,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization
0,-1.668620,-1.675547,-1.666923,-1.675847,-0.128616,0.043426,0.067413,0.500617,0.438943,-0.048217,...,-1.120053,-1.038348,-1.254581,1.796214,0.499973,-1.329357,-0.262246,-1.248318,1.698495,1.335001
1,-1.674196,-1.671910,-1.671093,-1.681444,-0.801205,0.019802,-0.019172,0.317097,-0.070093,-0.069058,...,-1.120053,-1.038348,-1.254581,1.796214,0.499973,-1.329357,-0.262246,-1.248318,1.698495,1.335001
2,-1.663216,-1.673221,-1.664566,-1.677733,-0.674991,0.020294,0.017197,0.548031,0.423097,-0.052112,...,-1.120053,-1.038348,-1.254581,1.796214,0.499973,-1.329357,-0.262246,-1.248318,1.698495,1.335001
3,-1.655972,-1.657837,-1.649033,-1.660824,-0.450393,0.018294,-0.028183,0.692781,1.240594,-0.024504,...,-1.120053,-1.038348,-1.254581,1.796214,0.499973,-1.329357,-0.262246,-1.248318,1.698495,1.335001
4,-1.655397,-1.660520,-1.653747,-1.664182,-0.814824,-0.001258,-0.083315,0.704498,0.953321,-0.013233,...,-1.120053,-1.038348,-1.254581,1.796214,0.499973,-1.329357,-0.262246,-1.248318,1.698495,1.335001


In [48]:
newDF=pd.concat([dateColumn,df,df_news_text,df_news_summary],axis=1)


In [49]:
newDF.shape

(974, 76)

In [50]:
newDF.head(5)

,datetime,close,open,low,high,volume,evm,force_index,rsi,cci,...,Spca16,Spca17,Spca18,Spca19,Spca20,Spca21,Spca22,Spca23,Spca24,Spca25
0,2019-02-20,-1.668620,-1.675547,-1.666923,-1.675847,-0.128616,0.043426,0.067413,0.500617,0.438943,...,1.818383,-0.914117,-2.074325,0.913988,0.112712,1.926086,0.260297,-1.321530,0.044715,1.352779
1,2019-02-21,-1.674196,-1.671910,-1.671093,-1.681444,-0.801205,0.019802,-0.019172,0.317097,-0.070093,...,1.650529,-0.181376,0.587466,-1.511634,1.054598,-1.483433,-1.453117,-0.526334,-0.115460,-1.474181
2,2019-02-22,-1.663216,-1.673221,-1.664566,-1.677733,-0.674991,0.020294,0.017197,0.548031,0.423097,...,-0.941116,0.429186,-1.367047,0.464688,-0.060432,0.348901,0.847478,0.569073,0.267420,0.057089
3,2019-02-25,-1.655972,-1.657837,-1.649033,-1.660824,-0.450393,0.018294,-0.028183,0.692781,1.240594,...,0.295991,-0.981859,0.993470,-1.462695,-0.706416,-0.579468,0.472940,0.470021,0.762850,-1.404587
4,2019-02-26,-1.655397,-1.660520,-1.653747,-1.664182,-0.814824,-0.001258,-0.083315,0.704498,0.953321,...,1.060050,0.447202,-2.514552,2.650107,-2.557092,-0.502712,2.301711,0.643496,-0.231621,-0.870302


Start applying model after this.

Take data before 2022-9-30 as training and after as testing.


In [51]:
file_train = newDF[(newDF['datetime'] <'2022-01-01')]
file_test = newDF[(newDF['datetime'] >='2022-01-01')]
y_df_train = y_df[(y_df['datetime'] <'2022-01-01')]
y_df_test = y_df[(y_df['datetime'] >='2022-01-01')]
print("train x: ",file_train.shape,"train y: ",y_df_train.shape)
print("test x: ",file_test.shape,"test y: ",y_df_test.shape)



train x:  (724, 76) train y:  (724, 2)
test x:  (250, 76) test y:  (250, 2)


In [52]:
file_train = file_train.drop('datetime', axis=1)
# file_train = file_train.drop('title', axis=1)
file_test = file_test.drop('datetime', axis=1)
y_df_train = y_df_train.drop('datetime', axis=1)
y_df_test = y_df_test.drop('datetime', axis=1)
file_train

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Spca16,Spca17,Spca18,Spca19,Spca20,Spca21,Spca22,Spca23,Spca24,Spca25
0,-1.668620,-1.675547,-1.666923,-1.675847,-0.128616,0.043426,0.067413,0.500617,0.438943,-0.048217,...,1.818383,-0.914117,-2.074325,0.913988,0.112712,1.926086,0.260297,-1.321530,0.044715,1.352779
1,-1.674196,-1.671910,-1.671093,-1.681444,-0.801205,0.019802,-0.019172,0.317097,-0.070093,-0.069058,...,1.650529,-0.181376,0.587466,-1.511634,1.054598,-1.483433,-1.453117,-0.526334,-0.115460,-1.474181
2,-1.663216,-1.673221,-1.664566,-1.677733,-0.674991,0.020294,0.017197,0.548031,0.423097,-0.052112,...,-0.941116,0.429186,-1.367047,0.464688,-0.060432,0.348901,0.847478,0.569073,0.267420,0.057089
3,-1.655972,-1.657837,-1.649033,-1.660824,-0.450393,0.018294,-0.028183,0.692781,1.240594,-0.024504,...,0.295991,-0.981859,0.993470,-1.462695,-0.706416,-0.579468,0.472940,0.470021,0.762850,-1.404587
4,-1.655397,-1.660520,-1.653747,-1.664182,-0.814824,-0.001258,-0.083315,0.704498,0.953321,-0.013233,...,1.060050,0.447202,-2.514552,2.650107,-2.557092,-0.502712,2.301711,0.643496,-0.231621,-0.870302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,1.595814,1.527659,1.580393,1.554807,-0.688898,1.786220,0.868584,0.862153,0.986832,-0.143126,...,-1.348111,0.013094,0.202696,-0.228257,0.259967,0.291705,-0.137759,0.458343,-0.255312,0.598122
720,1.571284,1.600886,1.615689,1.576252,-0.608764,1.593279,0.454527,0.707009,0.906312,-0.057541,...,0.664806,-0.215095,1.006945,0.182568,-0.245883,0.613784,-0.223907,0.452797,-1.003748,0.330362
721,1.573407,1.581089,1.606261,1.559756,-0.927342,1.058239,0.133907,0.714561,0.637582,-0.053188,...,4.536310,0.733698,-1.868503,-3.433487,-1.594664,1.632244,-2.006359,-3.219759,-2.049082,-1.532502
722,1.545575,1.584428,1.605052,1.558342,-0.976202,0.890338,0.090565,0.523835,0.423435,-0.204403,...,-1.731146,1.021985,1.190620,1.504136,0.151652,-0.923247,0.089130,-0.337751,0.956670,0.907953


In [53]:
file_test

,close,open,low,high,volume,evm,force_index,rsi,cci,macd-signal,...,Spca16,Spca17,Spca18,Spca19,Spca20,Spca21,Spca22,Spca23,Spca24,Spca25
724,1.635440,1.545310,1.595865,1.612779,-0.128042,0.637190,0.465917,0.838662,0.735515,-0.193051,...,0.966060,-0.679832,0.123816,-1.970873,-0.838483,1.107797,-2.106236,0.665066,-2.408328,0.372906
725,1.580955,1.659802,1.629952,1.614193,-0.226249,1.196772,0.361399,0.462909,0.543095,-0.301427,...,-0.447437,-1.647548,0.504535,-0.479987,-1.655417,0.026204,0.824727,0.751534,0.823216,0.291274
726,1.468209,1.587767,1.521646,1.548915,-0.316781,0.384472,-0.451278,-0.209672,-0.259246,-0.804068,...,1.640600,-2.829193,-1.326733,1.925505,-0.255668,-2.274602,1.217723,1.646906,-0.700347,-1.139291
727,1.399336,1.422947,1.449120,1.434149,-0.271894,-0.082995,-0.117687,-0.560674,-0.935698,-1.372400,...,0.710886,1.753162,-0.970937,-1.120789,1.117059,-1.780029,-0.589923,0.170238,-0.360842,0.333469
728,1.403345,1.427479,1.434373,1.406813,-0.465273,0.079685,-0.019464,-0.539856,-1.074780,-1.694447,...,-1.315761,-1.001168,0.238707,0.242876,2.077984,1.655151,2.048571,1.135930,-1.441565,-1.901032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,0.479385,0.508205,0.449704,0.474073,-0.633274,-2.369341,-1.142599,-1.601605,-1.238043,-1.503718,...,1.727583,2.827416,1.035260,-1.005180,-0.452188,1.798969,-0.634948,-1.344085,-0.228949,-2.238541
970,0.470608,0.426391,0.433748,0.423642,-0.899534,-2.908863,-0.909306,-1.634109,-1.214319,-1.413523,...,2.306204,-2.243432,-0.066597,1.084826,-2.327940,-1.495850,0.988852,-0.155374,0.065605,1.730080
971,0.427193,0.437363,0.411507,0.399840,-0.801034,-2.066408,-0.862952,-1.795609,-1.217095,-1.403124,...,0.873453,-2.523095,-0.288524,-1.318568,0.659262,-0.309576,1.374979,-1.640017,-0.738817,0.848646
972,0.332535,0.396576,0.342606,0.390885,-0.489376,-1.931024,-1.194451,-2.111416,-1.328902,-1.618963,...,0.050719,-2.594426,0.332053,-0.011782,-1.045049,-0.123244,2.000252,0.192048,0.394147,0.018608


In [54]:
y_df_train

,y_actual
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
719,0.0
720,1.0
721,0.0
722,0.0


In [55]:
y_df_test

,y_actual
724,0.0
725,0.0
726,0.0
727,1.0
728,1.0
...,...
969,0.0
970,0.0
971,0.0
972,1.0


In [56]:
# # file_train.columns = file_train.iloc[0]
# file_train = file_train.iloc[1:]

# # file_test.columns = file_test.iloc[0]
# file_test = file_test.iloc[1:]


# # y_df_train.columns = y_df_train.iloc[0]
# y_df_train = y_df_train.iloc[1:]


# # y_df_test.columns = y_df_test.iloc[0]
# y_df_test = y_df_test.iloc[1:]

In [57]:
print(y_df_test)

     y_actual
724       0.0
725       0.0
726       0.0
727       1.0
728       1.0
..        ...
969       0.0
970       0.0
971       0.0
972       1.0
973       1.0

[250 rows x 1 columns]


In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report




baseline_LR = LogisticRegression(C = 1e10, tol=0.000000001, max_iter=100000)
baseline_LR.fit(file_train, y_df_train)
    # print('Coefficients: \n', baseline_LR.coef_)
print("training accracy: %.4f" % baseline_LR.score(file_train, y_df_train))
print("test accuracy: %.4f" % baseline_LR.score(file_test, y_df_test))
y_fit_LR = baseline_LR.predict(file_test)
# print(y_fit_LR)
# print(y_df_test)
print('Accuracy:', accuracy_score(y_df_test, y_fit_LR))
print('ROC AUC Score:', roc_auc_score(y_df_test, y_fit_LR))
print('F1 score:', f1_score(y_df_test, y_fit_LR))
print()
print('\n clasification report:\n', classification_report(y_df_test, y_fit_LR))
print('\n confussion matrix:\n',confusion_matrix(y_df_test, y_fit_LR))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training accracy: 0.6478
test accuracy: 0.5440
Accuracy: 0.544
ROC AUC Score: 0.5410963305700148
F1 score: 0.5043478260869566


 clasification report:
               precision    recall  f1-score   support

         0.0       0.57      0.59      0.58       133
         1.0       0.51      0.50      0.50       117

    accuracy                           0.54       250
   macro avg       0.54      0.54      0.54       250
weighted avg       0.54      0.54      0.54       250


 confussion matrix:
 [[78 55]
 [59 58]]


In [59]:
from sklearn import svm

In [60]:
# Fit and report SVM
    # c = 0.5
svm_rbf = svm.SVC(C=0.5, kernel='rbf',max_iter=50000)
svm_rbf.fit(file_train, y_df_train)
# Training acc
pred = svm_rbf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the SVM is ', accuracy)
# Test acc
y_hat = svm_rbf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the SVM is ', test_accuracy)
# Conf matrix
print('\n Training classification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test classification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

The training accuracy of the SVM is  0.6174033149171271
The test accuracy of the SVM is  0.508

 Training classification report:
               precision    recall  f1-score   support

         0.0       0.80      0.20      0.32       327
         1.0       0.59      0.96      0.73       397

    accuracy                           0.62       724
   macro avg       0.70      0.58      0.53       724
weighted avg       0.69      0.62      0.55       724


 confusion matrix:
 [[ 66 261]
 [ 16 381]]

 Test classification report:
               precision    recall  f1-score   support

         0.0       0.62      0.19      0.29       133
         1.0       0.49      0.87      0.62       117

    accuracy                           0.51       250
   macro avg       0.56      0.53      0.46       250
weighted avg       0.56      0.51      0.45       250


 confusion matrix:
 [[ 25 108]
 [ 15 102]]


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [61]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
# Fit Random Forest
rf = RandomForestClassifier(n_estimators=10, oob_score=True, random_state=0)
rf.fit(file_train, y_df_train)
# Training acc
pred = rf.predict(file_train)
accuracy = accuracy_score(pred, y_df_train)
print('The training accuracy of the Random Forest is ', accuracy)
# Test acc
y_hat = rf.predict(file_test)
test_accuracy = accuracy_score(y_hat, y_df_test)
print('The test accuracy of the Random Forest is ', test_accuracy)
# Conf matrix
print('\n Training clasification report:\n', classification_report(y_df_train, pred))
print('\n confusion matrix:\n',confusion_matrix(y_df_train, pred))
print('\n Test clasification report:\n', classification_report(y_df_test, y_hat))
print('\n confusion matrix:\n',confusion_matrix(y_df_test, y_hat))

The training accuracy of the Random Forest is  0.9834254143646409
The test accuracy of the Random Forest is  0.472

 Training clasification report:
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       327
         1.0       1.00      0.97      0.98       397

    accuracy                           0.98       724
   macro avg       0.98      0.98      0.98       724
weighted avg       0.98      0.98      0.98       724


 confusion matrix:
 [[326   1]
 [ 11 386]]

 Test clasification report:
               precision    recall  f1-score   support

         0.0       0.50      0.53      0.52       133
         1.0       0.43      0.40      0.42       117

    accuracy                           0.47       250
   macro avg       0.47      0.47      0.47       250
weighted avg       0.47      0.47      0.47       250


 confusion matrix:
 [[71 62]
 [70 47]]


<ipython-input-62-30f767e98b6f>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(file_train, y_df_train)
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


In [67]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop,Adam
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [64]:
m1, n1 = file_train.shape
x_train = np.array(file_train).reshape(m1, n1, 1)
m2, n2 = file_test.shape
x_test= np.array(file_test).reshape(m2, n2, 1)

In [65]:
train_confusion = np.zeros((2, 2))
test_confusion = np.zeros((2, 2))

batch_size = 128
epochs = 80
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
print(x_train.shape)
# print(filename)
# if x_train.shape[0] == 0:
#     continue

# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             

# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('Training loss:', train_eval[0])
print('Training accuracy:', train_eval[1])
pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
train_confusion += train_conf

# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:',test_eval[1])
y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
test_confusion += test_conf
print(test_conf)
    

(724, 75, 1)
Epoch 1/80
6/6 [==============================] - 1s 24ms/step - loss: 0.7187 - accuracy: 0.5138
Epoch 2/80
6/6 [==============================] - 0s 21ms/step - loss: 0.6929 - accuracy: 0.5180
Epoch 3/80
6/6 [==============================] - 0s 22ms/step - loss: 0.6924 - accuracy: 0.5290
Epoch 4/80
6/6 [==============================] - 0s 21ms/step - loss: 0.6992 - accuracy: 0.5359
Epoch 5/80
6/6 [==============================] - 0s 23ms/step - loss: 0.6925 - accuracy: 0.5290
Epoch 6/80
6/6 [==============================] - 0s 21ms/step - loss: 0.6896 - accuracy: 0.5525
Epoch 7/80
6/6 [==============================] - 0s 22ms/step - loss: 0.6902 - accuracy: 0.5359
Epoch 8/80
6/6 [==============================] - 0s 22ms/step - loss: 0.6899 - accuracy: 0.5290
Epoch 9/80
6/6 [==============================] - 0s 24ms/step - loss: 0.6892 - accuracy: 0.5622
Epoch 10/80
6/6 [==============================] - 0s 21ms/step - loss: 0.6889 - accuracy: 0.5387
Epoch 11/80
6/6 

In [68]:
# train_confusion = np.zeros((2, 2))
# test_confusion = np.zeros((2, 2))

batch_size = 32
epochs = 200
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
print(x_train.shape)


# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             

# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('CNN training loss:', train_eval[0])
print('CNN training accuracy:', train_eval[1])
pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
# train_confusion += train_conf
print("Training confusion matrix \n",train_conf)

# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('CNN test loss:', test_eval[0])
print('CNN test accuracy:',test_eval[1])
y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
# test_confusion += test_conf
print("Testing confusion matrix \n",test_conf)
    

(724, 75, 1)
Epoch 1/200
23/23 [==============================] - 1s 10ms/step - loss: 0.7019 - accuracy: 0.5262
Epoch 2/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6893 - accuracy: 0.5318
Epoch 3/200
23/23 [==============================] - 0s 8ms/step - loss: 0.6911 - accuracy: 0.5456
Epoch 4/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6955 - accuracy: 0.5552
Epoch 5/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6945 - accuracy: 0.5483
Epoch 6/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6908 - accuracy: 0.5373
Epoch 7/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6895 - accuracy: 0.5428
Epoch 8/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6867 - accuracy: 0.5428
Epoch 9/200
23/23 [==============================] - 0s 9ms/step - loss: 0.6874 - accuracy: 0.5511
Epoch 10/200
23/23 [==============================] - 0s 8ms/step - loss: 0.6865 - accuracy: 0.

In [69]:
# train_confusion = np.zeros((2, 2))
# test_confusion = np.zeros((2, 2))

batch_size = 16
epochs = 500
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=x_train.shape[1:3]))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(3))

#model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
#model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=x_train.shape[1:3]))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
print(x_train.shape)


# Train model
history = model.fit(x_train, y_df_train,batch_size=batch_size,   epochs=epochs,verbose=1)
                
             

# Training report
train_eval = model.evaluate(x_train, y_df_train, verbose=0)
print('CNN training loss:', train_eval[0])
print('CNN training accuracy:', train_eval[1])
pred = model.predict(x_train)
pred = (pred > 0.5)
train_conf = confusion_matrix(y_df_train, pred)
# train_confusion += train_conf
print("Training confusion matrix \n",train_conf)

# Test report
test_eval = model.evaluate(x_test, y_df_test, verbose=0)
print('CNN test loss:', test_eval[0])
print('CNN test accuracy:',test_eval[1])
y_hat = model.predict(x_test)
y_hat = (y_hat > 0.5)
test_conf = confusion_matrix(y_df_test, y_hat)
# test_confusion += test_conf
print("Testing confusion matrix \n",test_conf)
    

(724, 75, 1)
Epoch 1/500
46/46 [==============================] - 1s 6ms/step - loss: 0.6978 - accuracy: 0.5207
Epoch 2/500
46/46 [==============================] - 0s 5ms/step - loss: 0.6900 - accuracy: 0.5539
Epoch 3/500
46/46 [==============================] - 0s 6ms/step - loss: 0.6901 - accuracy: 0.5401
Epoch 4/500
46/46 [==============================] - 0s 6ms/step - loss: 0.6941 - accuracy: 0.5525
Epoch 5/500
46/46 [==============================] - 0s 6ms/step - loss: 0.6988 - accuracy: 0.5069
Epoch 6/500
46/46 [==============================] - 0s 6ms/step - loss: 0.6909 - accuracy: 0.5359
Epoch 7/500
46/46 [==============================] - 0s 5ms/step - loss: 0.6902 - accuracy: 0.5318
Epoch 8/500
46/46 [==============================] - 0s 6ms/step - loss: 0.6895 - accuracy: 0.5414
Epoch 9/500
46/46 [==============================] - 0s 6ms/step - loss: 0.6879 - accuracy: 0.5483
Epoch 10/500
46/46 [==============================] - 0s 5ms/step - loss: 0.6861 - accuracy: 0.5